In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import seaborn as sns
from IPython.display import display
import os, sys, itertools, csv
pd.options.display.max_columns = 100

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.genome import get_feature_hit_set, is_overlap
from util.operon import get_operon_name_set, get_operon_ID_set
from util.feature import get_feat_d_from_ID
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD

In [2]:
exp_muts_df = pd.read_pickle("./data/7_df.pkl")

# exp_muts_df = pd.read_pickle("./data/2_4_df.pkl")  # debug
# exp_muts_df = exp_muts_df[exp_muts_df.exp=="SER"]
# exp_muts_df = exp_muts_df[exp_muts_df["mutation target annotation"]=="grxB/mdtH"]

display(len(exp_muts_df), exp_muts_df.head())

46

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
1232,4,R121C (CGC→TGC),xylR,SNP,3735339,NaN,C→T,1,SSW_XYL,116,1,1.0,1,True,"(3735339, 3735339)",{ECK120002449},"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",False,False,{},{},{},{},{},True,"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",{'ECK120002449': ['ECK120002449']},"[{'name': 'xylFGHR', 'RegulonDB ID': 'ECK12001...",{'ECK120014482': ['ECK120002449']},"[{'name': 'Fis', 'significantly associated con...","{'ArcA': ['xylFGHR'], 'Fis': ['xylFGHR'], 'CRP...",37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1233,5,None,rph,DEL,3815859,NaN,Δ82 bp,1,SSW_XYL,116,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1234,6,L770R (CTC→CGC),rpoC,SNP,4187658,NaN,T→G,1,SSW_XYL,116,1,1.0,1,True,"(4187658, 4187658)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1235,9,G164A (GGC→GCC),dnaG,SNP,3211597,NaN,G→C,2,SSW_XYL,109,1,1.0,1,True,"(3211597, 3211597)",{ECK120000233},"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},{},True,"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",{'ECK120000233': ['ECK120000233']},"[{'name': 'rpsU-dnaG-rpoD', 'RegulonDB ID': 'E...",{'ECK120014515': ['ECK120000233']},"[{'name': 'LexA', 'significantly associated co...",{'LexA': ['rpsU-dnaG-rpoD']},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1236,5,None,rph,DEL,3815859,NaN,Δ82 bp,2,SSW_XYL,109,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)


In [3]:
operon_df = pd.read_pickle("./data/operon_df.pkl")
operon_df.head()

,OPERON_ID,OPERON_NAME,FIRSTGENEPOSLEFT,LASTGENEPOSRIGHT,REGULATIONPOSLEFT,REGULATIONPOSRIGHT,OPERON_STRAND,OPERON_INTERNAL_COMMENT,KEY_ID_ORG,range
0,ECK120011191,rob,4634441,4635310,4634441,4635382,reverse,NaN,ECK12,"(4634441, 4635382)"
1,ECK120011203,argP,3059753,3060646,3059684,3060680,forward,NaN,ECK12,"(3059684, 3060680)"
2,ECK120011329,lrhA,2405703,2406641,2405703,2406866,reverse,NaN,ECK12,"(2405703, 2406866)"
3,ECK120011490,rtcR,3558268,3559866,3558268,3559866,forward,NaN,ECK12,"(3558268, 3559866)"
4,ECK120011629,lrp,932595,933089,932265,933089,forward,NaN,ECK12,"(932265, 933089)"


In [4]:
TU_df = pd.read_csv("./data/RegulonDB10/transcription_unit.txt", sep="\t", comment='#', header=None)
TU_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "PROMOTER_ID",
    "TRANSCRIPTION_UNIT_NAME",
    "OPERON_ID",
    "TRANSCRIPTION_UNIT_NOTE",
    "TU_INTERNAL_COMMENT",
    "KEY_ID_ORG"]


TU_objects_df = pd.read_csv("./data/RegulonDB10/tu_objects_tmp.txt",
                            sep="\t",
                            comment='#',
                            header=None,
                            )

TU_objects_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "NUMTU",
    "TU_POSLEFT",
    "TU_POSRIGHT",
    "TU_TYPE",
    "TU_OBJECT_CLASS",
    "TU_OBJECT_ID",
    "TU_OBJECT_NAME",
    "TU_OBJECT_POSLEFT",
    "TU_OBJECT_POSRIGHT",
    "TU_OBJECT_STRAND",
    "TU_OBJECT_COLORCLASS",
    "TU_OBJECT_DESCRIPTION",
    "TU_OBJECT_SIGMA",
    "TU_OBJECT_EVIDENCE",
    "TU_OBJECT_RI_TYPE",
    "TU_OBJECT_TYPE",
    "EVIDENCE"
]

TU_objects_df["TU range"] = TU_objects_df.apply(
    lambda row: (row["TU_POSLEFT"], row["TU_POSRIGHT"]), axis=1)

TU_objects_df.head()

,TRANSCRIPTION_UNIT_ID,NUMTU,TU_POSLEFT,TU_POSRIGHT,TU_TYPE,TU_OBJECT_CLASS,TU_OBJECT_ID,TU_OBJECT_NAME,TU_OBJECT_POSLEFT,TU_OBJECT_POSRIGHT,TU_OBJECT_STRAND,TU_OBJECT_COLORCLASS,TU_OBJECT_DESCRIPTION,TU_OBJECT_SIGMA,TU_OBJECT_EVIDENCE,TU_OBJECT_RI_TYPE,TU_OBJECT_TYPE,EVIDENCE,TU range
0,ECK120008913,3,1825955,1832013,H,PM,ECK120009851,astCp1,1832013,1832013,R,NaN,NaN,Sigma70,Human inference of promoter position,NaN,predicted,NaN,"(1825955, 1832013)"
1,ECK120008913,3,1825955,1832013,H,GN,ECK120003528,astE,1825955,1826923,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
2,ECK120008913,3,1825955,1832013,H,GN,ECK120003529,astB,1826916,1828259,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
3,ECK120008913,3,1825955,1832013,H,GN,ECK120003532,astC,1830762,1831982,R,"0,0,255",nitrogen metabolism,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
4,ECK120008913,3,1825955,1832013,H,GN,ECK120003530,astD,1828256,1829734,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"


# significance

In [5]:
op_len_df = pd.read_pickle("./data/operon_length_df.pkl")
op_len_df.head()

,operon description,length
0,unknown,265905


In [6]:
from util.feature import get_feat_d

In [7]:
# a bit behind annotations operons, etc.
# Still uses same approach to counted, just not all data-types have been consolidated into the operon feat col

feat_col_name = "operons"
link_col_name = "operon links"
comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for i, r in exp_muts_df.iterrows():
    for f, links in r[link_col_name].items():
        
        f_d = dict()
        if f == "unknown":
            f_d["RegulonDB ID"] = "unknown"
            f_d["name"] = "unknown"
            f_d["range"] = (0, op_len_df[op_len_df["operon description"]=="unknown"].iloc[0]["length"])
        else:
            f_d = get_feat_d(RegulonDB_ID=f, json=r[feat_col_name])
        
        if f_d["RegulonDB ID"] not in comp_mut_df.index:
            f_len = f_d["range"][1] - f_d["range"][0] + 1
            df = pd.DataFrame({"name": f_d["name"], "length": f_len, "observed mutation count": len(links)},
                              index=[f_d["RegulonDB ID"]])  # "name" column just for visual inspection
            comp_mut_df = comp_mut_df.append(df, sort=False)
        else:
            comp_mut_df.loc[f, "observed mutation count"] += len(links)
            
comp_mut_df

,observed mutation count,name,length
ECK120014482,2,xylFGHR,5280.0
ECK120014627,12,rph-pyrE,1485.0
ECK120016992,3,rplKAJL-rpoBC,11376.0
ECK120014515,1,rpsU-dnaG-rpoD,4317.0
ECK120020905,1,fimAICDFGH,7075.0
ECK120028518,1,eutHA,2627.0
ECK120014638,1,rpsMKD-rpoA-rplQ,3064.0
ECK120016920,2,rhoL-rho,1515.0
ECK120034085,1,ybeZYX-lnt,4245.0
ECK120029041,1,fabI,927.0


In [8]:
for _, r in exp_muts_df.iterrows():
    for feat_d in r["operons"]:
        feat_d["mutation set count"] = comp_mut_df.loc[feat_d["RegulonDB ID"]]["observed mutation count"]

In [9]:
mutated_seq_len = np.sum(comp_mut_df["length"])

# XXX Is this correct. What about overlapping operons?
# They would make the genome remainder smaller than what it should be.
TOTAL_GENOME_LEN = 4641652
genome_remainder_len = TOTAL_GENOME_LEN - mutated_seq_len

# Not account for the number of mutations that don't land in operons.
df = pd.DataFrame([[0, genome_remainder_len]], columns=["observed mutation count", "length"], index=["genome remainder"])
comp_mut_df = comp_mut_df.append(df, sort=False)
total_seq_len = TOTAL_GENOME_LEN


comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df.head()

,observed mutation count,name,length,proportion
ECK120014482,2,xylFGHR,5280.0,0.001138
ECK120014627,12,rph-pyrE,1485.0,0.000320
ECK120016992,3,rplKAJL-rpoBC,11376.0,0.002451
ECK120014515,1,rpsU-dnaG-rpoD,4317.0,0.000930
ECK120020905,1,fimAICDFGH,7075.0,0.001524


In [10]:
from collections import Counter
comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.
num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=num_muts,
        replace=True,
        p=proportion_l
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,observed mutation count,name,length,proportion,equal or larger count,p value,significant
ECK120014482,2,xylFGHR,5280.0,0.001138,17,0.0017,True
ECK120014627,12,rph-pyrE,1485.0,0.000320,0,0.0000,True
ECK120016992,3,rplKAJL-rpoBC,11376.0,0.002451,4,0.0004,True
ECK120014515,1,rpsU-dnaG-rpoD,4317.0,0.000930,449,0.0449,True
ECK120020905,1,fimAICDFGH,7075.0,0.001524,780,0.0780,False


In [11]:
from statsmodels.stats import multitest

pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

# pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,observed mutation count,name,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
ECK120014482,2,xylFGHR,5280.0,0.001138,17,0.0017,True,0.0578,False
ECK120014627,12,rph-pyrE,1485.0,0.000320,0,0.0000,True,0.0000,True
ECK120016992,3,rplKAJL-rpoBC,11376.0,0.002451,4,0.0004,True,0.0136,True
ECK120014515,1,rpsU-dnaG-rpoD,4317.0,0.000930,449,0.0449,True,1.0000,False
ECK120020905,1,fimAICDFGH,7075.0,0.001524,780,0.0780,False,1.0000,False


In [12]:
# not wanting to consider mutation events with only 1 features as being significant
# comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
comp_mut_df.head()

,observed mutation count,name,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
ECK120014482,2,xylFGHR,5280.0,0.001138,17,0.0017,True,0.0578,False
ECK120014627,12,rph-pyrE,1485.0,0.000320,0,0.0000,True,0.0000,True
ECK120016992,3,rplKAJL-rpoBC,11376.0,0.002451,4,0.0004,True,0.0136,True
ECK120014515,1,rpsU-dnaG-rpoD,4317.0,0.000930,449,0.0449,True,1.0000,False
ECK120020905,1,fimAICDFGH,7075.0,0.001524,780,0.0780,False,1.0000,False


In [13]:
for i, r in exp_muts_df.iterrows():
    for d in r["operons"]:
        if d["RegulonDB ID"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["RegulonDB ID"]]["corrected significance"]
#             d["significant"] = comp_mut_df.loc[d["RegulonDB ID"]]["final significance"]

# Outputting dataframe for the next step in the pipeline

In [14]:
exp_muts_df.to_pickle("./data/8_df.pkl")

In [15]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_ID(RegulonDB_ID, json):
#     name = ""
#     is_signif = False
#     for d in json:
#         if d["RegulonDB ID"] == RegulonDB_ID:
#             name = d["name"]
#             is_signif = d["significant"]
#             break
#     return (name, is_signif)


# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["operon links"]
#     for targ_ID, hits in link_d.items():
#         t, is_signif = _get_name_signif_from_ID(targ_ID, r["operons"])
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0", width=1)
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.1, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 operon mutated feature count")
# ax.get_legend().remove()

In [16]:
# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["operon links"]
#     for targ_ID, hits in link_d.items():
#         t, is_signif = _get_name_signif_from_ID(targ_ID, r["operons"])
#         if is_signif:
#             t = '*' + t
#             if t not in targ_cnt_d.keys():
#                 targ_cnt_d[t] = 0
#             targ_cnt_d[t] += len(hits)
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,13),
#                   color="#4C72B0", width=1)
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.35, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Significant operons")
# ax.get_legend().remove()